## This notebook contains the filled in code for ShortcourseExampleBase.ipynb

In [ ]:
# Import necessary modules 
from earthscopestraintools.mseed_tools import ts_from_mseed
from earthscopestraintools.gtsm_metadata import GtsmMetadata
from earthscopestraintools.timeseries import plot_timeseries_comparison

# Establish logging session
import logging
logger = logging.getLogger()
logging.basicConfig(
        format="%(message)s", level=logging.INFO
    )


## Aquire data and metadata


In [ ]:
# Get metadata and data

# First, set parameters for the seed code
network = 'PB' 
station = 'B073'
location = 'T0' # The location code for strain is T0
sample_rate = 'LS*' # LS = 1 Hz, * denotes data from all 4 gauges should be acquired

start = '2012-05-05T00:00:00.00'
end = '2012-05-10T00:00:00.00'

# Get the metadata
meta = GtsmMetadata(network=network,station=station)

# Get the data from the IRIS DMC 
strain_raw = ts_from_mseed(network=network,station=station,location=location,channel=sample_rate,start=start,end=end)


In [ ]:
# Examine the metadata contents through the .show() call 
meta.show()

In [ ]:
# Run this code cellto look at available attributes and functions of strain_raw
print('Attributes:')
funcs = []
print(list(vars(strain_raw).keys()))
for s in dir(strain_raw): 
    if s.startswith('_') != True and s not in vars(strain_raw).keys(): funcs.append(s)
print('Functions')
print(funcs)


In [ ]:
# Preview the data with the attribute
strain_raw.data

In [ ]:
# Use one of the functions to plot the data
strain_raw.plot()

## Filter and decimate to 5 minutes


In [ ]:
strain_raw5min = strain_raw.decimate_1s_to_300s()

## Linearization


In [ ]:
# Linearize
gauge_microstrain = strain_raw5min.linearize(reference_strains=meta.reference_strains,gap=meta.gap)
# Preview the data
gauge_microstrain.data

## Calculate regional strains

In [ ]:
# Which strain matrices are available in the metadata?
meta.strain_matrices

In [ ]:
# Apply both strain matrices to the gauge microstrain
lab_strain = gauge_microstrain.apply_calibration_matrix(meta.strain_matrices['lab'])
tide_strain = gauge_microstrain.apply_calibration_matrix(meta.strain_matrices['ER2010'])

In [ ]:
# Use a tool to plot a comparison of the matrices
plot_timeseries_comparison([lab_strain,tide_strain],names=['lab','Roeloffs 2010 tide'],zero=True)

## Signal correction

### Barometric Pressure Correction


In [ ]:
# First, get the atmospheric pressure data
atmp = ts_from_mseed(network=network,station=station,location='TS',channel='RDO',
                     scale_factor=0.001,start=start,end=end) # scale factor

In [ ]:
# Plot the correction
atmp.plot()

In [ ]:
# Examine the pressure response coefficients per gauge
# in microstrain/hPa
meta.atmp_response

In [ ]:
# Interpolate the pressure data to the timeseries
atmp_interp = atmp.interpolate(new_index=gauge_microstrain.data.index, series='hpa')

In [ ]:
# Calculate the pressure correction
atmp_c = atmp_interp.calculate_pressure_correction(response_coefficients=meta.atmp_response)

In [ ]:
# Plot the applied corrections
# The .apply_corrections() function is useful for this purpose
plot_timeseries_comparison([gauge_microstrain,gauge_microstrain.apply_corrections([atmp_c])],
                           ['Original','Pressure Corrected'],zero=True)

### Tide Correction


In [ ]:
# Take a look at the metadata for the tides
meta.tidal_params

In [ ]:
# Calculate tidal corrections for each gauge
tide_c = gauge_microstrain.calculate_tide_correction(tidal_parameters=meta.tidal_params,
                                                     longitude=meta.longitude)

### Trend Correction


In [ ]:
# Calculate the trend correction for each gauge
# Sometimes correcting for pressure first is the best bet
# because the pressure can have its own trend
trend_c = gauge_microstrain.apply_corrections([atmp_c]).linear_trend_correction(method='linear')


In [ ]:
# Plot various applied corrections
p1 = gauge_microstrain.apply_corrections([trend_c])
p2 = gauge_microstrain.apply_corrections([trend_c,atmp_c])
gauge_strain_corr = gauge_microstrain.apply_corrections([trend_c,atmp_c,tide_c])

plot_timeseries_comparison([gauge_microstrain,p1,p2,gauge_strain_corr],zero=True,
                           names=['Original','+Trend Corrected','+Pressure Corrected','+Tide Corrected'])

### Offset correction


In [ ]:
# Calculate offsets via first differencing above a cutoff limit 
# adjusted to the noise of the data

offset_c = gauge_strain_corr.calculate_offsets(limit_multiplier=100,cutoff_percentile=0.75)


In [ ]:
# Plot the offset corrections
# try adjusting the offset calculation parameters in the previous cell to see their effect

plot_timeseries_comparison([gauge_strain_corr,gauge_strain_corr.apply_corrections([offset_c])],
                           names=['Gauge Strain','Offsets Removed'])

**Whether you choose to apply these offsets is up to you, depending on your purpose forusing the data.**

## Tectonic or Nontectonic?



In [ ]:
# First, let's transform the corrected gauge strain to regional strain
reg_strain_corr = gauge_strain_corr.apply_calibration_matrix(meta.strain_matrices['ER2010'])

In [ ]:
# Get the rainfall data
rain = ts_from_mseed(network=network,station=station,location='TS',channel='RRO',start=start,end=end,scale_factor=0.0001)

In [ ]:
# Plot the rainfall with the corrected regional strains
# Look for residual pressure as well
%matplotlib inline
import matplotlib.pyplot as plt
plt.close('all')
reg_strain_corr.plot(rainfall=rain,atmp=atmp)

## Strain axes


In [ ]:
# Strain gif
%matplotlib widget
reg_strain_corr.strain_video(start='2012-05-08T20:00:00',end='2012-05-08T23:00:00',
                             interval=100)

## Include modeling example?